In [0]:
from datetime import datetime

In [0]:
%sql

CREATE TABLE SPARK_STREAMING_MCD_EX (ORDER_ID INT, ORDER_ITEM STRING, ORDER_TIME TIMESTAMP, BRANCH_ID STRING)
USING DELTA


In [0]:
%sql
INSERT INTO SPARK_STREAMING_MCD_EX VALUES
(1234,"Cheeseburger",TIMESTAMP'19:25',"SG_1"),
(1235,"French Fries",TIMESTAMP'19:28',"SG_1"),
(1236,"Cheeseburger",TIMESTAMP'19:29',"SG_2"),
(1237,"Chicken Nuggets",TIMESTAMP'19:31',"SG_1"),
(1238,"Grilled Chicken Wrap",TIMESTAMP'19:33',"SG_2")

num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql

DESCRIBE HISTORY SPARK_STREAMING_MCD_EX

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2023-04-17T14:01:18.000+0000,3818739110978553,venkatkarthick15@outlook.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(480196411471824),0417-115812-l5poife9,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 5, numOutputBytes -> 1553)",null,Databricks-Runtime/11.3.x-scala2.12
0,2023-04-17T13:54:50.000+0000,3818739110978553,venkatkarthick15@outlook.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(480196411471824),0417-115812-l5poife9,null,WriteSerializable,true,Map(),null,Databricks-Runtime/11.3.x-scala2.12


In [0]:
%sql

SELECT * FROM SPARK_STREAMING_MCD_EX


ORDER_ID,ORDER_ITEM,ORDER_TIME,BRANCH_ID
1234,Cheeseburger,2023-04-17T19:25:00.000+0000,SG_1
1235,French Fries,2023-04-17T19:28:00.000+0000,SG_1
1236,Cheeseburger,2023-04-17T19:29:00.000+0000,SG_2
1237,Chicken Nuggets,2023-04-17T19:31:00.000+0000,SG_1
1238,Grilled Chicken Wrap,2023-04-17T19:33:00.000+0000,SG_2


In [0]:
(spark.readStream
     .table("SPARK_STREAMING_MCD_EX")
     .createOrReplaceTempView("SPARK_STREAMING_MCD_VW")
)

In [0]:
%sql

SELECT * FROM SPARK_STREAMING_MCD_VW

ORDER_ID,ORDER_ITEM,ORDER_TIME,BRANCH_ID
1234,Cheeseburger,2023-04-17T19:25:00.000+0000,SG_1
1235,French Fries,2023-04-17T19:28:00.000+0000,SG_1
1236,Cheeseburger,2023-04-17T19:29:00.000+0000,SG_2
1237,Chicken Nuggets,2023-04-17T19:31:00.000+0000,SG_1
1238,Grilled Chicken Wrap,2023-04-17T19:33:00.000+0000,SG_2
1239,Cheeseburger,2023-04-17T19:25:00.000+0000,SG_1
1240,French Fries,2023-04-17T19:28:00.000+0000,SG_1
1241,Cheeseburger,2023-04-17T19:29:00.000+0000,SG_2


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW VW_SPARK_STREAMING_MCD AS (
  SELECT BRANCH_ID, ORDER_ITEM, COUNT(ORDER_ITEM) AS CNT FROM SPARK_STREAMING_MCD_VW
  GROUP BY BRANCH_ID, ORDER_ITEM
  ORDER BY BRANCH_ID
  )


In [0]:
%sql

select * from VW_SPARK_STREAMING_MCD

BRANCH_ID,ORDER_ITEM,CNT
SG_1,Chicken Nuggets,1
SG_1,French Fries,2
SG_1,Cheeseburger,2
SG_2,Cheeseburger,2
SG_2,Grilled Chicken Wrap,1


In [0]:
(spark.table("VW_SPARK_STREAMING_MCD")
      .writeStream
      .trigger(processingTime='5 seconds')
      .outputMode("complete")
      .option("checkpointLocation", "dbfs:/mnt/demo/spark_streaming_checkpoint")
      .table("mcd_spark_streaming_tbl")
)

Out[78]: <pyspark.sql.streaming.query.StreamingQuery at 0x7fc9b355cc10>

In [0]:
%sql

SELECT * FROM mcd_spark_streaming_tbl

BRANCH_ID,ORDER_ITEM,CNT
SG_2,Cheeseburger,1
SG_2,Grilled Chicken Wrap,1
SG_1,Chicken Nuggets,1
SG_1,French Fries,1
SG_1,Cheeseburger,1


Insert New values to the table

In [0]:
%sql
INSERT INTO SPARK_STREAMING_MCD_EX VALUES
(1239,"Cheeseburger",TIMESTAMP'19:25',"SG_1"),
(1240,"French Fries",TIMESTAMP'19:28',"SG_1"),
(1241,"Cheeseburger",TIMESTAMP'19:29',"SG_2")

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
SELECT * FROM mcd_spark_streaming_tbl

BRANCH_ID,ORDER_ITEM,CNT
SG_1,Chicken Nuggets,1
SG_1,French Fries,2
SG_1,Cheeseburger,2
SG_2,Cheeseburger,2
SG_2,Grilled Chicken Wrap,1
